<a href="https://colab.research.google.com/github/yshan9/deep_learning/blob/master/ml_hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Attribution of sources:
Course DS 6202 at the George Washington University by Professor Yuxiao Huang

https://github.com/yuxiaohuang/teaching/blob/master/gwu/machine_learning_I/fall_2020/code/p2_shallow_learning/p2_c1_data_preprocessing/code_example/.ipynb_checkpoints/classification-checkpoint.ipynb

https://github.com/yuxiaohuang/teaching/blob/master/gwu/machine_learning_I/fall_2020/code/utilities/p2_shallow_learning/pmlm_utilities_shallow.ipynb


In [21]:
from google.colab import drive
import sys

# Mount Google Drive
drive.mount('/content/drive')

# Get the absolute path of the current folder
abspath_curr = '/content/drive/My Drive/Colab Notebooks/teaching/gwu/machine_learning_I/fall_2020/code/p2_shallow_learning/p2_c1_data_preprocessing/code_example/'

# Get the absolute path of the shallow utilities folder
abspath_util_shallow = '/content/drive/My Drive/Colab Notebooks/teaching/gwu/machine_learning_I/fall_2020/code/utilities/p2_shallow_learning/'

# Get the absolute path of the shallow models folder
abspath_model_shallow = '/content/drive/My Drive/Colab Notebooks/teaching/gwu/machine_learning_I/fall_2020/code/models/p2_shallow_learning/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
# Change working directory to the absolute path of the shallow utilities folder
%cd $abspath_util_shallow

# Import the shallow utitilities
%run pmlm_utilities_shallow.ipynb

/content/drive/My Drive/Colab Notebooks/teaching/gwu/machine_learning_I/fall_2020/code/utilities/p2_shallow_learning


In [25]:
import warnings

# Ignore warnings
warnings.filterwarnings('ignore')

In [24]:
import matplotlib.pyplot as plt
%matplotlib inline 

# Set matplotlib sizes
plt.rc('font', size=20)
plt.rc('axes', titlesize=20)
plt.rc('axes', labelsize=20)
plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)
plt.rc('legend', fontsize=20)
plt.rc('figure', titlesize=20)

In [26]:
# The magic below allows us to use tensorflow version 2.x
%tensorflow_version 2.x 
import tensorflow as tf
from tensorflow import keras

In [27]:
# The random seed
random_seed = 42

# Set random seed in tensorflow
tf.random.set_seed(random_seed)

# Set random seed in numpy
import numpy as np
np.random.seed(random_seed)

In [2]:
#Define

import pandas as pd
import numpy as np


In [28]:
import pandas as pd

# Load the raw data
df_test = pd.read_csv('/content/drive/My Drive/ds6202/new_york_city_taxi_trip_duration/test.csv', header=0
                )

# Remove the unnamed column
#df_raw = df_raw.drop(columns='Unnamed: 32')

# Make a copy of df_raw
df = df_test.copy(deep=True)

# Get the name of the target
target = 'diagnosis'

In [2]:
# Print the dimension of df
pd.DataFrame([[df.shape[0], df.shape[1]]], columns=['# rows', '# columns'])

,# rows,# columns
0,625134,9


In [3]:
# Print the first 5 rows of df
df.head()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N
1,id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N
2,id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N
3,id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,N
4,id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,N


In [12]:
# Load the raw data
df_train = pd.read_csv('/content/drive/My Drive/ds6202/new_york_city_taxi_trip_duration/train.csv', header=0
                )

# Remove the unnamed column
#df_train = df_train.drop(columns='Unnamed: 32')

# Make a copy of df_raw
df2 = df_train.copy(deep=True)

In [29]:
#Splitting the Data
from sklearn.model_selection import train_test_split

# Divide the data into training (60%) and test (40%)
df_train, df_test = train_test_split(df, 
                                     train_size=0.6, 
                                  
                                   )
#random_state=random_seed
#stratify=df[target]

# Divide the test data into validation (50%) and test (50%)
df_val, df_test = train_test_split(df_test, 
                                   train_size=0.5, 
                                   
                                   )
#random_state=random_seed, 
#stratify=df_test[target]

# Reset the index
df_train, df_val, df_test = df_train.reset_index(drop=True), df_val.reset_index(drop=True), df_test.reset_index(drop=True)

In [16]:
# Print the dimension of df_train
pd.DataFrame([[df_train.shape[0], df_train.shape[1]]], columns=['# rows', '# columns'])

,# rows,# columns
0,375080,9


In [7]:
# Print the dimension of df_val
pd.DataFrame([[df_val.shape[0], df_val.shape[1]]], columns=['# rows', '# columns'])

,# rows,# columns
0,125027,9


In [8]:
# Print the dimension of df_test
pd.DataFrame([[df_test.shape[0], df_test.shape[1]]], columns=['# rows', '# columns'])

,# rows,# columns
0,125027,9


In [30]:
# Handling Uncommon Features

# Call common_var_checker
df_common_var = common_var_checker(df_train, df_val, df_test, target)

# Print df_common_var
df_common_var


,common var
0,dropoff_latitude
1,dropoff_longitude
2,id
3,passenger_count
4,pickup_datetime
5,pickup_latitude
6,pickup_longitude
7,store_and_fwd_flag
8,vendor_id


In [31]:
# Get the features in the training data but not in the validation or test data
uncommon_feature_train_not_val_test = np.setdiff1d(df_train.columns, df_common_var['common var'])

# Print the uncommon features
pd.DataFrame(uncommon_feature_train_not_val_test, columns=['uncommon feature'])

,uncommon feature


In [32]:
# Get the features in the validation data but not in the training or test data
uncommon_feature_val_not_train_test = np.setdiff1d(df_val.columns, df_common_var['common var'])

# Print the uncommon features
pd.DataFrame(uncommon_feature_val_not_train_test, columns=['uncommon feature'])

,uncommon feature


In [33]:
# Get the features in the test data but not in the training or validation data
uncommon_feature_test_not_train_val = np.setdiff1d(df_test.columns, df_common_var['common var'])

# Print the uncommon features
pd.DataFrame(uncommon_feature_test_not_train_val, columns=['uncommon feature'])

,uncommon feature


In [34]:
#Removing uncommon features

# Remove the uncommon features from the training data
df_train = df_train.drop(columns=uncommon_feature_train_not_val_test)

# Print the first 5 rows of df_train
df_train.head()



,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,id0199975,1,2016-06-20 14:44:13,1,-73.969521,40.785561,-73.963547,40.772831,N
1,id2354218,1,2016-01-01 03:46:36,1,-73.971695,40.756836,-73.940910,40.672909,N
2,id3609986,1,2016-05-26 17:33:23,1,-73.789841,40.647053,-73.789841,40.647053,N
3,id0727860,1,2016-04-28 09:58:41,1,-73.957809,40.761292,-73.976524,40.737720,N
4,id0945154,2,2016-02-29 08:22:28,1,-73.987556,40.750000,-73.963287,40.762589,N


In [35]:
# Remove the uncommon features from the validation data
df_val = df_val.drop(columns=uncommon_feature_val_not_train_test)

# Print the first 5 rows of df_val
df_val.head()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,id3594406,1,2016-06-22 00:26:31,1,-73.993698,40.767212,-74.000877,40.740017,N
1,id0105471,1,2016-03-05 16:33:45,1,-73.990982,40.738037,-74.016846,40.704777,N
2,id1564733,1,2016-04-07 07:24:42,1,-73.993973,40.751354,-73.981987,40.758156,N
3,id2753941,2,2016-06-03 10:39:09,1,-73.960411,40.775749,-73.950592,40.773338,N
4,id2454335,2,2016-02-01 20:00:20,1,-73.969711,40.761650,-73.979721,40.765751,N


In [36]:
# Remove the uncommon features from the test data
df_test = df_test.drop(columns=uncommon_feature_test_not_train_val)

# Print the first 5 rows of df_test
df_test.head()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,id1960509,1,2016-05-27 22:10:15,1,-73.996864,40.720608,-73.994698,40.751839,N
1,id3541125,2,2016-03-09 23:42:12,1,-73.782013,40.644642,-73.793968,40.667301,N
2,id2714076,1,2016-05-25 08:34:50,1,-74.008148,40.738865,-73.984024,40.756577,N
3,id3349656,1,2016-05-26 12:59:43,1,-73.995026,40.725121,-73.993881,40.720901,N
4,id0163950,2,2016-04-20 09:53:34,6,-73.958168,40.784672,-73.962669,40.773022,N


In [43]:
#Handling identifiers


# Combine df_train, df_val and df_test
df = pd.concat([df_train, df_val, df_test], sort=False)

In [44]:
# Call id_checker on df
# See the implementation in pmlm_utilities.ipynb
df_id = id_checker(df)

# Print the first 5 rows of df_id
df_id.head()

""
0
1
2
3
4


In [45]:
#Removing identifies 

import numpy as np

# Remove identifiers from df_train
df_train.drop(columns=np.intersect1d(df_id.columns, df_train.columns), inplace=True)

# Remove identifiers from df_val
df_val.drop(columns=np.intersect1d(df_id.columns, df_val.columns), inplace=True)

# Remove identifiers from df_test
df_test.drop(columns=np.intersect1d(df_id.columns, df_test.columns), inplace=True)

In [46]:
# Print the first 5 rows of df_train
df_train.head()

,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,1,2016-06-20 14:44:13,1,-73.969521,40.785561,-73.963547,40.772831,N
1,1,2016-01-01 03:46:36,1,-73.971695,40.756836,-73.940910,40.672909,N
2,1,2016-05-26 17:33:23,1,-73.789841,40.647053,-73.789841,40.647053,N
3,1,2016-04-28 09:58:41,1,-73.957809,40.761292,-73.976524,40.737720,N
4,2,2016-02-29 08:22:28,1,-73.987556,40.750000,-73.963287,40.762589,N


In [47]:
# Print the first 5 rows of df_val
df_val.head()

,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,1,2016-06-22 00:26:31,1,-73.993698,40.767212,-74.000877,40.740017,N
1,1,2016-03-05 16:33:45,1,-73.990982,40.738037,-74.016846,40.704777,N
2,1,2016-04-07 07:24:42,1,-73.993973,40.751354,-73.981987,40.758156,N
3,2,2016-06-03 10:39:09,1,-73.960411,40.775749,-73.950592,40.773338,N
4,2,2016-02-01 20:00:20,1,-73.969711,40.761650,-73.979721,40.765751,N


In [48]:
# Print the first 5 rows of df_test
df_test.head()

,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,1,2016-05-27 22:10:15,1,-73.996864,40.720608,-73.994698,40.751839,N
1,2,2016-03-09 23:42:12,1,-73.782013,40.644642,-73.793968,40.667301,N
2,1,2016-05-25 08:34:50,1,-74.008148,40.738865,-73.984024,40.756577,N
3,1,2016-05-26 12:59:43,1,-73.995026,40.725121,-73.993881,40.720901,N
4,2,2016-04-20 09:53:34,6,-73.958168,40.784672,-73.962669,40.773022,N


In [49]:
#Handling date time variables

# Get the date time variables
datetime_vars = []

In [50]:
# Call datetime_transformer on df_train
df_train = datetime_transformer(df_train, datetime_vars)

# Print the first 5 rows of df_train
df_train.head()

,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,1,2016-06-20 14:44:13,1,-73.969521,40.785561,-73.963547,40.772831,N
1,1,2016-01-01 03:46:36,1,-73.971695,40.756836,-73.940910,40.672909,N
2,1,2016-05-26 17:33:23,1,-73.789841,40.647053,-73.789841,40.647053,N
3,1,2016-04-28 09:58:41,1,-73.957809,40.761292,-73.976524,40.737720,N
4,2,2016-02-29 08:22:28,1,-73.987556,40.750000,-73.963287,40.762589,N


In [52]:
# Call datetime_transformer on df_val
df_val = datetime_transformer(df_val, datetime_vars)

# Print the first 5 rows of df_val
df_val.head()

,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,1,2016-06-22 00:26:31,1,-73.993698,40.767212,-74.000877,40.740017,N
1,1,2016-03-05 16:33:45,1,-73.990982,40.738037,-74.016846,40.704777,N
2,1,2016-04-07 07:24:42,1,-73.993973,40.751354,-73.981987,40.758156,N
3,2,2016-06-03 10:39:09,1,-73.960411,40.775749,-73.950592,40.773338,N
4,2,2016-02-01 20:00:20,1,-73.969711,40.761650,-73.979721,40.765751,N


In [53]:
# Call datetime_transformer on df_test
df_test = datetime_transformer(df_test, datetime_vars)

# Print the first 5 rows of df_test
df_test.head()

,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,1,2016-05-27 22:10:15,1,-73.996864,40.720608,-73.994698,40.751839,N
1,2,2016-03-09 23:42:12,1,-73.782013,40.644642,-73.793968,40.667301,N
2,1,2016-05-25 08:34:50,1,-74.008148,40.738865,-73.984024,40.756577,N
3,1,2016-05-26 12:59:43,1,-73.995026,40.725121,-73.993881,40.720901,N
4,2,2016-04-20 09:53:34,6,-73.958168,40.784672,-73.962669,40.773022,N


In [54]:
#Handling missing data

# Combine df_train, df_val and df_test
df = pd.concat([df_train, df_val, df_test], sort=False)

In [55]:
# Call nan_checker on df
# See the implementation in pmlm_utilities.ipynb
df_nan = nan_checker(df)

# Print df_nan
df_nan

,var,proportion,dtype


In [56]:

# Print the unique data type of variables with NaN
pd.DataFrame(df_nan['dtype'].unique(), columns=['dtype'])

,dtype


In [57]:
# Get the variables with missing values, their proportion of missing values and data type
df_miss = df_nan[df_nan['dtype'] == 'float64'].reset_index(drop=True)

# Print df_miss
df_miss

,var,proportion,dtype


In [58]:
#Separating the training, validation and test data

# Separating the training data
df_train = df.iloc[:df_train.shape[0], :]

# Separating the validation data
df_val = df.iloc[df_train.shape[0]:df_train.shape[0] + df_val.shape[0], :]

# Separating the test data
df_test = df.iloc[df_train.shape[0] + df_val.shape[0]:, :]

In [59]:
# Print the dimension of df_train
pd.DataFrame([[df_train.shape[0], df_train.shape[1]]], columns=['# rows', '# columns'])

,# rows,# columns
0,375080,8


In [60]:
# Print the dimension of df_val
pd.DataFrame([[df_val.shape[0], df_val.shape[1]]], columns=['# rows', '# columns'])

,# rows,# columns
0,125027,8


In [61]:
# Print the dimension of df_test
pd.DataFrame([[df_test.shape[0], df_test.shape[1]]], columns=['# rows', '# columns'])

,# rows,# columns
0,125027,8


In [62]:
#Removing missing values

# If there are missing values
if len(df_miss['var']) > 0:
    # Remove rows with missing values from df_train
    df_remove_train = df_train.dropna(subset=np.intersect1d(df_miss['var'], df_train.columns),
                                    inplace=False)

    # Remove rows with missing values from df_val
    df_remove_val = df_val.dropna(subset=np.intersect1d(df_miss['var'], df_val.columns),
                                    inplace=False)

    # Remove rows with missing values from df_test
    df_remove_test = df_test.dropna(subset=np.intersect1d(df_miss['var'], df_test.columns),
                                inplace=False)

In [63]:

#Imputing missing values

from sklearn.impute import SimpleImputer

# If there are missing values
if len(df_miss['var']) > 0:
    # The SimpleImputer
    si = SimpleImputer(missing_values=np.nan, strategy='mean')

    # Impute the variables with missing values in df_train, df_val and df_test 
    df_train[df_miss['var']] = si.fit_transform(df_train[df_miss['var']])
    df_val[df_miss['var']] = si.transform(df_val[df_miss['var']])
    df_test[df_miss['var']] = si.transform(df_test[df_miss['var']])

In [64]:
#Encoding the data
#Combining the training, validation and test data


# Combine df_train, df_val and df_test
df = pd.concat([df_train, df_val, df_test], sort=False)

# Print the unique data type of variables in df
pd.DataFrame(df.dtypes.unique(), columns=['dtype'])


,dtype
0,int64
1,object
2,float64


In [2]:
# Identifying categorical variables

# Call cat_var_checker on df
# See the implementation in pmlm_utilities.ipynb
df_cat = cat_var_checker(df)

# Print the dataframe
df_cat

NameError: ignored

In [3]:
# Encoding categorical features

# One-hot-encode the categorical features in the combined data
df = pd.get_dummies(df, columns=np.setdiff1d(df_cat['var'], [target]))

# Print the first 5 rows of df
df.head()



NameError: ignored

In [2]:
# Encoding categorical target

from sklearn.preprocessing import LabelEncoder

# The LabelEncoder
le = LabelEncoder()

# Encode categorical target in the combined data
df[target] = le.fit_transform(df[target].astype(str))

# Print the first 5 rows of df
df.head()

NameError: ignored

In [2]:
# Separating the training, validation and test data

# Separating the training data
df_train = df.iloc[:df_train.shape[0], :]

# Separating the validation data
df_val = df.iloc[df_train.shape[0]:df_train.shape[0] + df_val.shape[0], :]

# Separating the test data
df_test = df.iloc[df_train.shape[0] + df_val.shape[0]:, :]

NameError: ignored

In [5]:
# Print the dimension of df_train
pd.DataFrame([[df_train.shape[0], df_train.shape[1]]], columns=['# rows', '# columns'])

NameError: ignored

In [ ]:
# Print the dimension of df_val
pd.DataFrame([[df_val.shape[0], df_val.shape[1]]], columns=['# rows', '# columns'])

In [ ]:
# Print the dimension of df_test
pd.DataFrame([[df_test.shape[0], df_test.shape[1]]], columns=['# rows', '# columns'])

In [ ]:
# Splitting the feature and target

# Get the feature matrix
X_train = df_train[np.setdiff1d(df_train.columns, [target])]
X_val = df_val[np.setdiff1d(df_val.columns, [target])]
X_test = df_test[np.setdiff1d(df_test.columns, [target])]

# Get the target vector
y_train = df_train[target]
y_val = df_val[target]

In [6]:
# Scaling the data

from sklearn.preprocessing import MinMaxScaler

# The MinMaxScaler
mms = MinMaxScaler()

In [7]:
# Normalize the training data
X_mms_train = pd.DataFrame(mms.fit_transform(X_train), columns=X_train.columns)

# Normalize the validation data
X_mms_val = pd.DataFrame(mms.transform(X_val), columns=X_val.columns)

# Normalize the test data
X_mms_test = pd.DataFrame(mms.transform(X_test), columns=X_test.columns)

NameError: ignored